In [ ]:
sc

## 02. Coded Frames using k-means dictionary

In [ ]:
val scale = 1/200.0
val samplesPerSecond = 360

val WINDOW = 120 // 32 or 40 samples per window
val WindowsPerFrame = 30 // 30 ,32 or 150
//val STEP = 2
//val SAMPLES = 200000

val samplesPerFrame = WindowsPerFrame * WINDOW
val secondPerFrame = samplesPerFrame / samplesPerSecond
val inputDataFile = "105s1.dat"
//val inputDataFile = "a02.dat"
val hdfsOutPath = "/user/fsainz/data/out/"


val framesFileName = "frames-"+ inputDataFile.split('.')(0)+"_" + secondPerFrame +"sLE.csv"
val ModelFileName = hdfsOutPath + "dict-" + inputDataFile.split('.')(0) + "_W" + WINDOW +".LEmodel"

var outputPath = "/user/fsainz/data/out/framesCoded-"+ inputDataFile.split('.')(0) + "_" +secondPerFrame  +"sLE/"

In [ ]:
// Databricks  
//val rootDir = "FileStore/fxo/ted/"   //local File
// GCP
val rootDir = "hdfs://spark-clu-m/user/fsainz/data/out/" 
val csvFileName = rootDir + framesFileName
println("Procesing: " + csvFileName)
println("With: " + ModelFileName)
println("Output Files: " + outputPath)

In [ ]:

import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint 



In [ ]:
var windowingF  =  new Array[Double](WINDOW)   //differs of Array[Double](WINDOW) uajj
for ( i <- 0 until WINDOW) {
    val y = Math.sin(Math.PI * i / (WINDOW - 1.0));
          windowingF(i)=  y * y
 }



### Till now there is a bug in proccesDataFrame function, processing contour of the frames (firts and last half window) so coded data is trunked!!

In [ ]:
def processDataframe ( wSize: Int, frame : Array[Double] , model:KMeansModel) : Array[Double] = {
  
  // Split frame in WINDOW size windows
  val frameLength = frame.length 
  
  assert( frameLength % wSize == 0 )  // should throws a exception if not
  
  val signalWindows = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]
  val codecWindows = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]
  val ret = scala.collection.mutable.ArrayBuffer.empty[Double]
  val nWindows:Int = frameLength / wSize 
  
  for ( i <- 0 until (nWindows * 2)-1) {   // we step 1/2 wSize
    var p = i*wSize/2
    var signalWindow = frame.slice(p, p+wSize  )
   
    var tapWindow = (signalWindow,windowingF).zipped.map(_*_)   // multiple vector item by item
    val wNorm = Vectors.norm(Vectors.dense(tapWindow) , 2)
    tapWindow = tapWindow.map( x => x/wNorm)   
   
    val nc = model.predict(Vectors.dense(tapWindow))
    
    val cluster = model.clusterCenters(nc)
   
    val codecWindow = cluster.toArray.map(x => x*wNorm)   // scale to signal 
 
    signalWindows.append(signalWindow)
    codecWindows.append(codecWindow.toArray)
    // Coded windows has tapering function scale. get it of it usign a trick sum two desplacements halfs. 
    // see propierties or tapering funtion.
    // we produce half a window a time
    // and we have to arrange in some way first and last halfs
    var half1 = new Array[Double](wSize)
    var half2 = new Array[Double](wSize)
    if (i == 0) {
      half1 = codecWindows(i).slice(0,wSize/2)  
      half2 = half1  
      print("+")
      val r = (half1 , half2).zipped.map(_+_)   // sum halfs
     // ret ++= r
    } 
    else {
      print("*")
      half1= codecWindows(i-1).slice(wSize/2,wSize)
      half2= codecWindows(i).slice(0,wSize/2)  
      val r = (half1 , half2).zipped.map(_+_)   // sum halfs
      ret ++= r
    }
    
    // last half window 
    if ( i == (nWindows*2) -2 ) {
      half1= codecWindows(i).slice(wSize/2,wSize)
      half2= half1 
      print ("-" )
      val r = (half1 , half2).zipped.map(_+_)   // sum halfs
      // ret ++= r
    }
  }
  ret.toArray
}

In [ ]:


val framesECGRdd = sc.textFile(csvFileName).map(x => x.split(",").map(_.toDouble) )

In [ ]:

val clusters = KMeansModel.load(sc, ModelFileName)


In [ ]:
val halfWindowsSize = WINDOW /2 

val top = samplesPerFrame-halfWindowsSize
top-halfWindowsSize

In [ ]:
framesECGRdd.first().slice(halfWindowsSize,samplesPerFrame-halfWindowsSize).length
framesECGRdd.first().slice(halfWindowsSize,top).length

In [ ]:
val framesCodedRdd = framesECGRdd.map( x => (x, processDataframe(WINDOW, x ,clusters ) )).map(x => (x._1.slice(halfWindowsSize,top ), x._2, (x._1.slice(halfWindowsSize,samplesPerFrame-halfWindowsSize),x._2).zipped.map(_-_)))

In [ ]:
framesCodedRdd.first._2.length

In [ ]:

framesCodedRdd.map(x => (x._1.mkString(",") + ";" + x._2.mkString(",") + ";" + x._3.mkString(","))).saveAsTextFile(outputPath)
println(" SAVED: " + outputPath)

In [ ]:
// It no posible to save directly as csv if we want to put several field data...//
///var outputFileName ="/user/fsainz/data/out/framesCoded-a02.csv"//
//framesCodedDF.write.format("com.databricks.spark.csv").option("header", "false").mode("overwrite").save(outputFileName)

In [ ]:
// it a pity, It is not possible to share tables with pySpark..
import org.apache.spark.sql.SQLContext
val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
framesCodedRdd.toDF.createOrReplaceTempView("framesProccesed")
framesCodedDF.createOrReplaceTempView("framesProccesed")